<a href="https://colab.research.google.com/github/MartynaPlomecka/DLM_DigitalFootprint/blob/main/updated__main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Tue Apr 26 13:16:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [3]:
## for data
import json
import pandas as pd
import numpy as np
from sklearn import metrics, manifold
## for processing
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for bert
!pip install transformers
!pip install datasets
import transformers
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments
import sys
import gzip
import datasets
from datetime import datetime

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
     |████████████████████████████████| 4.0 MB 8.7 MB/s 
     |████████████████████████████████| 895 kB 57.7 MB/s 
     |████████████████████████████████| 6.6 MB 55.6 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 77 kB 5.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 325 kB 9.0 MB/s 
     |████████████████████████████████| 1.1 MB 39.6 MB/s 
     |████████████████████████████████| 212 kB 67.1 MB/s 
     |████████████████████████████████| 136 kB 69.5 MB/s 
     |████████████████████████████████| 127 kB 64.6 MB/s 
     |████████████████████████████████| 144 kB 58.6 MB/s 

In [6]:
# mount to Google drive
from google.colab import drive
drive.mount('/content/drive')
# dataset paths
dataset_path = '/content/drive/My Drive/embeddings/table4model_clean.csv'
not_labelled_indeed_path = '/content/drive/My Drive/embeddings/indeed_intro.txt'



#not labeled data
dtf = pd.read_csv(not_labelled_indeed_path, header = 0)
dtf

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Introduction
0,"Creative problem-solver, who sees the big pict..."
1,To obtain a position as a medical assistant in...
2,Currently seeking employment as an entry level...
3,Around 3+ years of professional IT experience ...
4,Highly motivated sales and marketing represent...
...,...
99995,Work in an environment where my skills and ene...
99996,To work for an organization that allows me to ...
99997,Experienced and highly motivated Shipping & Re...
99998,Over six years of experience in customer servi...


# masked language modelling


Masked Language Model (MLM) is the process how BERT was pre-trained. MLM is a powerful pre-training strategy for learning sentence embeddings. This is especially the case when we work on a specialized domain.



In [7]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments
import sys
import gzip
from datetime import datetime
no_deprecation_warning=True

In [8]:
model_name = sys.argv[1]
per_device_train_batch_size = 64

save_steps = 1000               #Save model every 1k steps
num_train_epochs = 3            #Number of epochs
use_fp16 = False                #Set to True, if your GPU supports FP16 operations
max_length = 100                #Max length for a text input
do_whole_word_mask = True       #If set to true, whole words are masked
mlm_prob = 0.15                 #Probability that a word is replaced by a [MASK] token

# Load the model
model = AutoModelForMaskedLM.from_pretrained('distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')


Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [9]:
transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM

transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM

In [10]:
 
#output_dir = "output/{}-{}".format(model_name.replace("/", "_"),  datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
output_dir = "output/model"

print("Save checkpoints to:", output_dir)


Save checkpoints to: output/model


In [11]:
##### Load train dataset

train_sentences = []
train_path = '/content/drive/My Drive/embeddings/indeed_intro.txt'
with gzip.open(train_path, 'rt', encoding='utf8') if train_path.endswith('.gz') else  open(train_path, 'r', encoding='utf8') as fIn:
    for line in fIn:
        line = line.strip()
        if len(line) >= 10:
            train_sentences.append(line)

print("Train sentences:", len(train_sentences))

dev_sentences = []
if len(sys.argv) >= 4:
    dev_path = sys.argv[3]
    with gzip.open(dev_path, 'rt', encoding='utf8') if dev_path.endswith('.gz') else open(dev_path, 'r', encoding='utf8') as fIn:
        for line in fIn:
            line = line.strip()
            if len(line) >= 10:
                dev_sentences.append(line)

print("Dev sentences:", len(dev_sentences))

#A dataset wrapper, that tokenizes our data on-the-fly
class TokenizedSentencesDataset:
    def __init__(self, sentences, tokenizer, max_length, cache_tokenization=False):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.max_length = max_length
        self.cache_tokenization = cache_tokenization

    def __getitem__(self, item):
        if not self.cache_tokenization:
            return self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)

        if isinstance(self.sentences[item], str):
            self.sentences[item] = self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)
        return self.sentences[item]

    def __len__(self):
        return len(self.sentences)

train_dataset = TokenizedSentencesDataset(train_sentences, tokenizer, max_length)
dev_dataset = TokenizedSentencesDataset(dev_sentences, tokenizer, max_length, cache_tokenization=True) if len(dev_sentences) > 0 else None

##### Training arguments

if do_whole_word_mask:
    data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)
else:
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    evaluation_strategy="steps" if dev_dataset is not None else "no",
    per_device_train_batch_size=per_device_train_batch_size,
    eval_steps=save_steps,
    save_steps=save_steps,
    logging_steps=save_steps,
    save_total_limit=1,
    prediction_loss_only=True,
    fp16=use_fp16
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

print("Save tokenizer to:", output_dir)
tokenizer.save_pretrained(output_dir)

trainer.train()

print("Save model to:", output_dir)
model.save_pretrained(output_dir)

print("Training done")


Train sentences: 207389
Dev sentences: 0


tokenizer config file saved in output/model/tokenizer_config.json
Special tokens file saved in output/model/special_tokens_map.json
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 207389
  Num Epochs = 3
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 9723


Save tokenizer to: output/model


Step,Training Loss
1000,2.377300
2000,2.163100
3000,2.061200
4000,1.996900
5000,1.948300
6000,1.918200
7000,1.893600
8000,1.854600
9000,1.842800


Saving model checkpoint to output/model/checkpoint-1000
Configuration saved in output/model/checkpoint-1000/config.json
Model weights saved in output/model/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to output/model/checkpoint-2000
Configuration saved in output/model/checkpoint-2000/config.json
Model weights saved in output/model/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [output/model/checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to output/model/checkpoint-3000
Configuration saved in output/model/checkpoint-3000/config.json
Model weights saved in output/model/checkpoint-3000/pytorch_model.bin
Deleting older checkpoint [output/model/checkpoint-2000] due to args.save_total_limit
Saving model checkpoint to output/model/checkpoint-4000
Configuration saved in output/model/checkpoint-4000/config.json
Model weights saved in output/model/checkpoint-4000/pytorch_model.bin
Deleting older checkpoint [output/model/checkpoint-3000] due to args

Save model to: output/model


Model weights saved in output/model/pytorch_model.bin


Training done


In [ ]:
#data_path = '/content/drive/My Drive/embeddings/dev.txt'

# Model:

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
state_dict = torch.load("output/model/pytorch_model.bin") 
model.load_state_dict(state_dict)

<All keys matched successfully>

In [13]:
!nvidia-smi


Tue Apr 26 14:32:33 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P0    39W / 250W |   8995MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Labeled dataset:

In [20]:
dtf_l

,URL,proactive,grit,bias_to_action,resourcefulness,openness_to_experience,conscientiousness,agreeableness,extraversion,emotional_stability,...,emotionally_expressive,passive,creativity,easy-going,conventionality,calm,liveliness,spontaneity,impassiveness,About
0,https://www.linkedin.com/in/ugljesajovanovic/,-,100,75,50,-,-,-,-,50,...,50,-,75,50,25,-,-,-,-,A passionate Android developer who enjoys buil...
1,https://www.linkedin.com/in/miruna-carmen-barb...,-,75,25,75,72,63,78,75,50,...,50,-,50,25,50,-,-,-,-,Experienced Research Fellow with a demonstrate...
2,https://www.linkedin.com/in/nataliamaria-kwiat...,75,100,100,75,80,-,-,-,50,...,75,25,100,25,0,-,-,-,-,"CPQ Product owner, SAP Workstream Lead and Dig..."
3,https://www.linkedin.com/in/magda-wdowczyk-2ab...,-,25,75,25,88,63,66,75,50,...,100,-,75,75,25,-,-,-,-,I am a person who has many interests - fashion...
4,https://www.linkedin.com/in/jazm%C3%ADn-del-sa...,-,75,25,100,59,81,63,72,50,...,50,-,50,0,50,-,-,-,-,"Political scientist, specialized in Audiovisua..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
623,https://www.linkedin.com/in/%C5%82ukasz-krzemi...,0,50,100,33,48,57,55,68,-,...,17,83,50,67,67,59,67,-,33,"Aspiring Frontend developer, Homebrewing enthu..."
624,https://www.linkedin.com/in/halina-borcz,17,0,0,17,50,8,50,67,-,...,17,33,63,33,17,33,67,-,17,For many years I have been interested in widel...
625,https://www.linkedin.com/in/monika-sz,33,13,17,0,36,46,45,25,-,...,17,17,25,50,33,33,0,-,50,"Skills:- SQL, PostgreSQL - basic level,- Java ..."
626,https://www.linkedin.com/in/juan-martin-de-die...,0,75,67,33,17,46,50,75,-,...,33,83,38,33,33,67,67,-,33,"Bilingual Industrial Engineer, with a strong c..."


In [44]:
torch.cuda.empty_cache() # PyTorch thing


In [31]:
#labeled:
dtf_l = pd.read_csv(dataset_path, header = 0)
labeled = dtf_l[['About', 'grit', 'bias_to_action', 'resourcefulness']]
labeled

,About,grit,bias_to_action,resourcefulness
0,A passionate Android developer who enjoys buil...,100,75,50
1,Experienced Research Fellow with a demonstrate...,75,25,75
2,"CPQ Product owner, SAP Workstream Lead and Dig...",100,100,75
3,I am a person who has many interests - fashion...,25,75,25
4,"Political scientist, specialized in Audiovisua...",75,25,100
...,...,...,...,...
623,"Aspiring Frontend developer, Homebrewing enthu...",50,100,33
624,For many years I have been interested in widel...,0,0,17
625,"Skills:- SQL, PostgreSQL - basic level,- Java ...",13,17,0
626,"Bilingual Industrial Engineer, with a strong c...",75,67,33


In [35]:
# for debugging:
train = labeled[0:40]
val = labeled[40:55]
test= labeled[55:62]

#splits
train = labeled[0:400]
val = labeled[400:550]
test= labeled[550:]

#to json
labeled_json = labeled.to_json()
labeled.to_json('data.json', lines=True, orient='records')

train_json = train.to_json()
train.to_json('data_train.json', lines=True, orient='records')

val_json = val.to_json()
val.to_json('data_val.json', lines=True, orient='records')

test_json = test.to_json()
test.to_json('data_test.json', lines=True, orient='records')

from datasets import Dataset
dataset = Dataset.from_json('data.json')
train_dataset = Dataset.from_json('data_train.json')
val_dataset = Dataset.from_json('data_val.json')
test_dataset = Dataset.from_json('data_test.json')

Using custom data configuration default-0dd11ded7d767f2c


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-9ad28e221644105f


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-0dd11ded7d767f2c/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

...

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-ec018b2ae90d3312


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d4ae0e90c92455fb/0.0.0. Subsequent calls will reuse this data.


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-ec018b2ae90d3312/0.0.0. Subsequent calls will reuse this data.


In [ ]:
#model = AutoModelForSequenceClassification.from_pretrained(BASE_MODEL, id2label=id2label, label2id=label2id)


## Prepare Datasets
### 1.tokenising the dataset by calling tokenizer. 
### 2.associating the label attribute to each dataset item.


In [36]:
#this is done on labeled dataset in jsons

ds = {"train": train_dataset, "validation": val_dataset, "test": test_dataset}

def preprocess_function(examples):
    label = examples["grit"] 
    examples = tokenizer(examples["About"], truncation=True, padding="max_length", max_length=256)
    examples["label"] = float(label)
    return examples

for split in ds:
    ds[split] = ds[split].filter(lambda e: e['grit'].isnumeric())
    ds[split] = ds[split].map(preprocess_function, remove_columns=["About", "grit"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/397 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/149 [00:00<?, ?ex/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/78 [00:00<?, ?ex/s]

# Metrics
## We can compute metrics to track the model’s improvement during training.
## We retrieve the class with the highest logit (corresponding to the highest probability) for each prediction and compare it with the actual label to calculate the global accuracy score.

In [37]:
import numpy as np
from datasets import load_metric

#metric = load_metric("accuracy")

#def compute_metrics(eval_pred):
#    logits, labels = eval_pred
#    predictions = np.argmax(logits, axis=-1)
#    return metric.compute(predictions=predictions, references=labels)

We put the output directory for the trained model and the learning parameters into TrainingArguments. With load_best_model_at_end and metric_for_best_model, we will keep several best models (i.e. those with the highest accuracy on the validation set) during training and load the best model at the end.

In [38]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="../models/",
    learning_rate=1e-3,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=64,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    #metric_for_best_model="accuracy",
    load_best_model_at_end=True,
    weight_decay=0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [39]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)
    #print(np.array(labels).shape)
    #print(np.array(logits).shape)
    logits =[logits[i][0][0] for i in range(len(logits))] 
    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()

    return {"mse": mse, "mae": mae, "r2": r2}

In [ ]:
#tutaj zrob multioutput regression

In [40]:

import torch
torch.cuda.empty_cache()
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        #print(outputs[0][:, 0].size())
        #a = torch.nn.Linear(30522, 1).cuda()
        #logits = a(outputs[0][:, 0].cuda())
        logits = outputs[0][0, 0][0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

        torch.cuda.empty_cache()


In [41]:
trainer = RegressionTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["validation"],
    compute_metrics=compute_metrics_for_regression,
)

torch.cuda.empty_cache() # PyTorch thing
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: bias_to_action, resourcefulness. If bias_to_action, resourcefulness are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 397
  Num Epochs = 64
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 25408
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Using a target size (torch.Size([1])) that is different to the input 

Epoch,Training Loss,Validation Loss,Mse,Mae,R2
1,No log,1303.707397,1303.707397,30.481373,-0.878854
2,846.232700,1512.617920,1512.617920,32.977180,-1.179928
3,819.917000,1331.374756,1331.374756,30.831743,-0.918727
4,781.425000,1151.566895,1151.566895,28.509592,-0.659595
5,781.425000,1376.671387,1376.671387,31.393847,-0.984007
6,824.623000,1636.908569,1636.908569,34.452358,-1.359051
7,854.970400,1344.274048,1344.274048,30.993793,-0.937317
8,767.419600,1622.600830,1622.600830,34.268726,-1.338431
9,824.597200,1242.358643,1242.358643,29.719759,-0.790440
10,824.597200,1375.901855,1375.901855,31.384460,-0.982898


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: bias_to_action, resourcefulness. If bias_to_action, resourcefulness are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 149
  Batch size = 1


1
2
(149, 1)
(149, 256, 30522)
2
2
2
2


Saving model checkpoint to ../models/checkpoint-397
Configuration saved in ../models/checkpoint-397/config.json
Model weights saved in ../models/checkpoint-397/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if sys.path[0] == '':
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: bias_to_action, resourcefulness. If bias_to_action, resourcefulness are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 149
  Batch size = 1


...

1
2
(149, 1)
(149, 256, 30522)
2
2
2
2


Saving model checkpoint to ../models/checkpoint-25011
Configuration saved in ../models/checkpoint-25011/config.json
Model weights saved in ../models/checkpoint-25011/pytorch_model.bin
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if sys.path[0] == '':
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: bias_to_action, resourcefulness. If bias_to_action, resourcefulness are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 149
  Batch size = 1


1
2
(149, 1)
(149, 256, 30522)
2
2
2
2


Saving model checkpoint to ../models/checkpoint-25408
Configuration saved in ../models/checkpoint-25408/config.json
Model weights saved in ../models/checkpoint-25408/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ../models/checkpoint-4367 (score: 1021.9090576171875).


TrainOutput(global_step=25408, training_loss=836.3550604140428, metrics={'train_runtime': 1717.3804, 'train_samples_per_second': 14.795, 'train_steps_per_second': 14.795, 'total_flos': 1684056927043584.0, 'train_loss': 836.3550604140428, 'epoch': 64.0})

# Evaluation
Note that we rely on the validation set’s accuracy to retrieve the best model. Calling Trainer.evaluate(), we can retrieve the best accuracy attained during training


In [45]:
torch.cuda.empty_cache() 

trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: bias_to_action, resourcefulness. If bias_to_action, resourcefulness are not expected by `DistilBertForMaskedLM.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 149
  Batch size = 1
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  if sys.path[0] == '':


1
2
(149, 1)
(149, 256, 30522)
2
2
2
2


{'epoch': 64.0,
 'eval_loss': 1021.9090576171875,
 'eval_mae': 26.58478355407715,
 'eval_mse': 1021.9090576171875,
 'eval_r2': -0.4727367983568813,
 'eval_runtime': 16.7574,
 'eval_samples_per_second': 8.892,
 'eval_steps_per_second': 8.892}